In [1]:
import re
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import dateutil.parser as dateparser
import random
import json

In [2]:
pwd

'/home/hanna'

In [3]:

with open('result.json', 'w') as fp:
    practice_json = json.dump(practice_dict, fp)
print(practice_json)

NameError: name 'practice_dict' is not defined

In [4]:

URL =  "https://old.reddit.com/r/datascience/top/?sort=top&t=year"
#'https://old.reddit.com/r/datascience/'
    #'https://old.reddit.com/r/datascience/top/?sort=top&t=year'
      #'https://old.reddit.com/r/datascience/top/?sort=top&t=year&count=25&after=t3_eufeqm'
      #'https://old.reddit.com/r/datascience/top/?sort=top&t=year&count=50&after=t3_fo5stq'
      #'https://old.reddit.com/r/datascience/top/?sort=top&t=year&count=75&after=t3_cnvc3e'

USER_AGENT_LIST = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [69]:
tst = ['a', 'b']

user_tst = [x for x in USER_AGENT_LIST for _ in (0,1)]

In [101]:


def html_data(url, user_agent_list):
    
    users = []
    
    for i in range(1,2):
        
        print(f'Page #: {i} \n URL: {url}')
        
        user_agent = random.choice(user_agent_list)
        
        headers = {'User-Agent': user_agent}
        
        html = requests.get(url , headers = headers)
        print(url, '\n')
        
        soup = BeautifulSoup(html.text, 'html.parser')
        #print(soup.prettify())
        threads = soup.find_all('span', class_='domain')
        
        for thread in threads:
            if thread not in ['(self.datascience)', 'i.redd.it', 'v.redd.it']:
                continue
            parent_div = thread.parent.parent.parent.parent
        
        status_code = html.status_code
        
        attrs = {'class': 'thing', 'data-domain': ['self.datascience', 'i.redd.it', 'v.redd.it']}
        
        domains, domain_tag, titles, award_count, authors, author_info, comments, likes, dislikes, dates = ([] for i in range(10))
        
        for post in soup.find_all('div', attrs=attrs):
            
            domains.append(post.attrs['data-domain'])
            
            #titles.append(post.find('p', class_='title').text)   
            
            title_info = post.find('p', class_='title')
            
            #print(title_info.find('span', class_='crosspost-badge'))
            
            # This should be new title
            #print(post.find('div', class_='entry unvoted').div.p.text)
            print(post.find('div', class_='entry unvoted').div[:3])
            break
            
            # Count the awards 
            award_count = 0
            if post.find('span', class_='awardings-bar').a:
                awards = post.find('span', class_='awardings-bar').prettify()
                award_count_list = [ i.replace('data-count=', "").strip('"') for i in re.findall(r'data-count="\d+\"' , awards)]
                award_count_list = [int(i) for i in award_count_list]
                award_count = sum(award_count_list)
            #print(award_count)
            
            
            # Display any domain tags, otherwise show N/A
            if '(' not in title_info.span.text:
                domain_tag.append(title_info.span.text)
            else:
                domain_tag.append('N/A') # TODO: change this to a placeholder string object

            
            # TODO: switch over to a dictionary and use a .get instead ??
            try:
                authors.append(post.find('a', class_='author').text)
            except AttributeError:
                authors.append('Unknown')
                
            
            # Additional info on the author
            if post.find('span', class_='flair flair-seniorflair'):
                author_additional_info = post.find('span', class_='flair flair-seniorflair').text
            else:
                author_additional_info = 'N/A'
                
            author_info.append(author_additional_info)
            
            #comments.append(post.find('a', class_='comments').text.split()[0])
            #likes.append(post.find('div', attrs={'class': 'score likes'}).text)
            #dislikes.append(post.find('div', attrs={'class': 'score dislikes'}).text)
            
            # Date parsing, and regex to select datetime objecta
            #date = post.find('p', attrs={'class': 'tagline'})
            #date = date.time.prettify()
            #dates.append(re.findall(r'\d{4}\-\d{2}\-\w+\:\d+\:\d+\+\d+' , date)[0])
            
            
            #print(dateparser.parse(re.findall(r'\d{4}\-\d{2}\-\w+\:\d+\:\d+\+\d+' , date)[0]))
            #print(type(dateparser.parse(re.findall(r'\d{4}\-\d{2}\-\w+\:\d+\:\d+\+\d+' , date)[0])))
            #print('\n', dates[0].time.prettify())

            #print(post)
        users.append([user_agent, status_code, domain_tag, titles, award_count, authors, author_info, comments, likes, dislikes, dates])
        
        next_button = soup.find('span', class_='next-button')
        
        url = next_button.find('a').attrs['href']
     
        #print('\n \n Next page \n\n')
        
        time.sleep(2)
    
    return users

In [102]:
html_data(URL, USER_AGENT_LIST)


Page #: 1 
 URL: https://old.reddit.com/r/datascience/top/?sort=top&t=year


ConnectionError: HTTPSConnectionPool(host='old.reddit.com', port=443): Max retries exceeded with url: /r/datascience/top/?sort=top&t=year (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f3350fbe650>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [24]:
print(res)

[['Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)', 200, ['It’s never too early (i.redd.it)', 'Imposter Syndrome is a problem for me and I think this is the main contributor (i.redd.it)', 'Fun/TriviaTrue that (i.redd.it)', 'Fun/TriviaDS at a glance (i.redd.it)', 'Fun/TriviaGraph of graph analysis (i.redd.it)', 'When the boss doesn’t like your charts (i.redd.it)', "ToolingFor any python & pandas users out there, here's a free tool to visualize your dataframes (v.redd.it)", 'Fun/TriviaThe next time my coworkers ask what metrics I used for my model. (v.redd.it)', 'Fun/TriviaFit an exponential curve to anything... (i.redd.it)', "Beware of today's data (i.redd.it)", "Fun/TriviaIt's Meme Monday, so here's a python meme for DS folks (i.redd.it)", 'Fun/TriviaExpectation vs reality (i.redd.it)', 'Found this (i.redd.it)', 'Fun/TriviaWhen you get an Excel Sheet of 1000x5 and your clients ask you to do "Data Science" on this with "AI" (i.redd.it)', 'Fun/TriviaRecruiters be like (

In [89]:

html_data(URL, USER_AGENT_LIST)

Page #: 1 
 URL: https://old.reddit.com/r/datascience/top/?sort=top&t=year
https://old.reddit.com/r/datascience/top/?sort=top&t=year 

Page #: 2 
 URL: https://old.reddit.com/r/datascience/top/?sort=top&t=year&count=25&after=t3_eufeqm
https://old.reddit.com/r/datascience/top/?sort=top&t=year&count=25&after=t3_eufeqm 



AttributeError: 'NoneType' object has no attribute 'find'

In [58]:
practice_dict = {}

practice_dict['titles'] = res[0][2]

practice_dict['authors'] = res[0][3]

practice_dict['comments'] = res[0][4]

practice_dict['likes'] = res[0][5]

practice_dict['dislikes'] = res[0][6]

practice_dict['dates'] = res[0][7]

In [60]:
practice_dict['titles']


['It’s never too early (i.redd.it)',
 'Imposter Syndrome is a problem for me and I think this is the main contributor (i.redd.it)',
 'Fun/TriviaTrue that (i.redd.it)',
 'Fun/TriviaDS at a glance (i.redd.it)',
 'Fun/TriviaGraph of graph analysis (i.redd.it)',
 'When the boss doesn’t like your charts (i.redd.it)',
 "ToolingFor any python & pandas users out there, here's a free tool to visualize your dataframes (v.redd.it)",
 'Fun/TriviaThe next time my coworkers ask what metrics I used for my model. (v.redd.it)',
 'Fun/TriviaFit an exponential curve to anything... (i.redd.it)',
 "Beware of today's data (i.redd.it)",
 "Fun/TriviaIt's Meme Monday, so here's a python meme for DS folks (i.redd.it)",
 'Fun/TriviaExpectation vs reality (i.redd.it)',
 'Found this (i.redd.it)',
 'Fun/TriviaWhen you get an Excel Sheet of 1000x5 and your clients ask you to do "Data Science" on this with "AI" (i.redd.it)',
 'Fun/TriviaRecruiters be like (i.redd.it)',
 'The formatting struggle. (i.redd.it)',
 'Fun/T

In [79]:
a = {'b': []}
a['b'].append(1)

In [80]:
a

{'b': [1]}

In [81]:


def html_data(url, user_agent_list):
    
    users = []
    
    domains, titles, authors, comments, likes, dislikes, dates = ([] for i in range(7))
    
    for i in range(1,3):
        
        user_agent = random.choice(user_agent_list)
        
        headers = {'User-Agent': user_agent}
        
        html = requests.get(url, headers = headers)
        
        soup = BeautifulSoup(html.text, 'html.parser')
        
        threads = soup.find_all('span', class_='domain')
        
        for thread in threads:
            if thread not in ['(self.datascience)', 'i.redd.it', 'v.redd.it']:
                continue
            parent_div = thread.parent.parent.parent.parent
        
        status_code = html.status_code
        
        attrs = {'class': 'thing', 'data-domain': ['self.datascience', 'i.redd.it', 'v.redd.it']}
        
        for post in soup.find_all('div', attrs=attrs):
            
            domains.append(post.attrs['data-domain'])
            #print(post.find('p', class_='title').span)#.panprettify())
            titles.append(post.find('p', class_='title').text)
            
            # This is very bad practice! I will change it to .get when we switch over to a dictionary
            try:
                authors.append(post.find('a', class_='author').text)
            except AttributeError:
                authors.append('Unknown')
                
                
            comments.append(post.find('a', class_='comments').text.split()[0])
            likes.append(post.find('div', attrs={'class': 'score likes'}).text)
            dislikes.append(post.find('div', attrs={'class': 'score dislikes'}).text)
            
            # Date parsing, and regex to select datetime objecta
            date = post.find('p', attrs={'class': 'tagline'})
            date = date.time.prettify()
            dates.append(re.findall(r'\d{4}\-\d{2}\-\w+\:\d+\:\d+\+\d+' , date))
            #print('\n', dates[0].time.prettify())

            #print(post)
        users.append([user_agent, status_code, titles, authors, comments, likes, dislikes, dates])
        
        next_button = soup.find('span', class_='next-button')
        
        url = next_button.find('a').attrs['href']
        print(f'Page #: {i} \n URL: {url}')
        #print('\n \n Next page \n\n')
        
        time.sleep(2)
    
    return users

In [82]:
    domains, titles, authors, comments, likes, dislikes, dates = ([] for i in range(7))

In [47]:
user_agent = random.choice(USER_AGENT_LIST)
        
headers = {'User-Agent': user_agent}
        
START_URL = 'https://old.reddit.com/r/datascience/top/?sort=top&t=year'
page = requests.get(START_URL, headers)
soup = BeautifulSoup(page.text, 'html.parser')
attrs = {'class': 'thing', 'data-domain': ['self.datascience', 'i.redd.it', 'v.redd.it']}
results = []
for res in soup.findAll('div', attrs=attrs):
    info = [desc.strip() for desc in res.descendants if type(desc) == NavigableString]
    results.append({'Domains': info[0], 'Title': info[1], 'Author': info[3], 'Comments': info[4], 'Likes': info[5], 'Dislikes': info[6]})

print(results)


[]


In [115]:
def samesame(var):
    """
    This function adds the inputs elements together
    """
    output = var[0]
    for elem in var[1:]:
        output += elem
    return output

assert samesame([[1,2], [3,4]]) == [1, 2, 3, 4]
assert samesame([2, 4, 5]) == 11
assert samesame(['a', 'b', 'c']) == 'abc'

In [ ]:
http://code.google.com/
    /edu/languages/google-python-class/images/puzzle/a-babd.jpg